# Decoding Logits - Sandbox

In [1]:
%load_ext autoreload
%autoreload 2
import sys
if "../src" not in sys.path:
    sys.path.append("../src")

In [2]:
from datetime import datetime
from pathlib import Path
from itertools import islice
from tqdm.auto import tqdm
import numpy as np
import torch
from torch.utils.data import DataLoader

import datasets
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq
from transformers import MT5TokenizerFast
from vec4gloss import check_hashes
from vec4gloss import Vec4GlossModel

In [33]:
datasets.disable_caching()

In [4]:
ds_defgen = datasets.load_from_disk("../data/defgen_dataset_cwn")
vec4gloss_model_dir = "../data/models/vec4gloss-defgen-220629-1250"

In [5]:
tokenizer = MT5TokenizerFast.from_pretrained(vec4gloss_model_dir)

## Preprocess functions

In [6]:
max_length = 256

def get_marked_pos(text):
    assert text.count("<") == text.count(">") == 1
    s, e = text.index("<")+1, text.index(">")    
    assert s != e
    return s, e

def add_marked_pos(ex):
    pos = get_marked_pos(ex["src"])
    return {"decoder_start_markers": pos[0], "decoder_end_markers": pos[1]}

def preprocess_fn(batch):    
    src_batch = tokenizer(batch["src"], 
                          max_length=max_length, truncation=True)
    start_markers = [src_batch.char_to_token(bi,s) 
                     for bi, s in enumerate(batch["decoder_start_markers"])]
    end_markers = [src_batch.char_to_token(bi,e) 
                   for bi, e in enumerate(batch["decoder_end_markers"])]
    
    with tokenizer.as_target_tokenizer():
        tgt_batch = tokenizer(batch["tgt"],
                              max_length=max_length, truncation=True)        
        
    return {
        **src_batch, 
        "decoder_start_markers": start_markers,
        "decoder_end_markers": end_markers,
        "labels": tgt_batch["input_ids"]
    }

In [7]:
drop_columns = ["cwnid", "src", "tgt"]
ds_defgen = (ds_defgen.map(add_marked_pos)
             .map(preprocess_fn, batched=True, remove_columns=drop_columns))
train_ds = ds_defgen["train"]
test_ds = ds_defgen["test"]

Loading cached processed dataset at ../data/defgen_dataset_cwn/train\cache-73645a22723c5115.arrow
Loading cached processed dataset at ../data/defgen_dataset_cwn/test\cache-79eff688f816c6ef.arrow


  0%|          | 0/77 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

## Model

In [8]:
torch.manual_seed(12345)
model = Vec4GlossModel.from_pretrained(vec4gloss_model_dir)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest")

In [9]:
g_cuda = torch.Generator()
g_cuda.manual_seed(211321)
loader = DataLoader(train_ds, batch_size=2, collate_fn=data_collator, shuffle=True, generator=g_cuda)
batches = list(islice(loader, 10))

In [10]:
# labels
batch = batches[0]
tokenizer.batch_decode(torch.where(batch["labels"]>=0, batch["labels"], 0))

['VC。模仿或照原樣重製他人的創意當作自己的。</s>',
 'Nc。美術館的建築物及建築物所在的位置。</s><pad><pad><pad><pad><pad>']

In [11]:
## direct decoding settings
with torch.no_grad():    
    batch = batches[0]
    out = model(**batch)
tokenizer.batch_decode(out.logits.argmax(2))

['VC。比仿特定模原來的的製後的文字意。作自己的。</s>', 'Nc。美術館的建築物及建築物所在的位置。</s></s> N N N N']

In [12]:
## generation setting
dbg = {}
batch = batches[0]
gen_batch = {k:v for k, v in batch.items() if k not in ("labels", "decoder_input_ids")}
tokenizer.batch_decode(model.generate(**gen_batch))

['<pad> VC。比喻自行取得資料。</s><pad><pad><pad><pad><pad>',
 '<pad> Nc。美術館的建築物及建築物所在的位置。</s>']

In [13]:
genout = model.generate(**gen_batch, return_dict_in_generate=True, 
                        output_scores=True, output_hidden_states=True, 
                        output_attentions=True
                       )

In [14]:
list(genout.keys())

['sequences',
 'scores',
 'encoder_attentions',
 'encoder_hidden_states',
 'decoder_attentions',
 'cross_attentions',
 'decoder_hidden_states']

In [15]:
tokenizer.batch_decode(torch.vstack([genout.scores[i].argmax(1) for i in range(len(genout.scores))]).permute(1,0))

['VC。比喻自行取得資料。</s>字 VC VC VC VC', 'Nc。美術館的建築物及建築物所在的位置。</s>']

In [16]:
intext = "我們<上>山途中欣賞沿途風景。"
vbatch = tokenizer(intext, return_tensors="pt")
s,e = get_marked_pos(intext)
s = vbatch.char_to_token(s)
e = vbatch.char_to_token(e)
vbatch["decoder_start_markers"] = torch.tensor([s])
vbatch["decoder_end_markers"] = torch.tensor([e])

In [17]:
vgenout = model.generate(**vbatch, max_length=30, 
                         return_dict_in_generate=True,
                         output_scores=True, output_hidden_states=True, 
                        output_attentions=True)
tokenizer.batch_decode(vgenout.sequences)

['<pad> VCL。往高處移動或去。</s>']

In [18]:
list(vgenout.keys())

['sequences',
 'scores',
 'encoder_attentions',
 'encoder_hidden_states',
 'decoder_attentions',
 'cross_attentions',
 'decoder_hidden_states']

In [19]:
## this is what I expect if decoding only uses a single vector
## It's a tuple(gen length) of tuple (decoder layers) of tensor
## (batch_size, num_heads, sequence_length, sequence_length)
vgenout.cross_attentions[4][7].shape

torch.Size([1, 12, 1, 1])

## split the encoder decoder

In [19]:
def extract_encoder_vector(intext, tokenizer, model):    
    vbatch = tokenizer(intext, return_tensors="pt")
    s,e = get_marked_pos(intext)   
    s = vbatch.char_to_token(s)
    e = vbatch.char_to_token(e)
    vbatch["decoder_start_markers"] = torch.tensor([s])
    vbatch["decoder_end_markers"] = torch.tensor([e])
    encoder = model.get_encoder()
    enc_out = encoder(
            input_ids=vbatch["input_ids"], 
            attention_mask=vbatch["attention_mask"])
    enc_vec = enc_out.last_hidden_state[[0],s:e,:] \
                     .mean(1, keepdim=True)
    return enc_vec

def decode_vector(vec, tokenizer, model, max_length=50):
    vgenout = model.generate(decoder_encoder_vector=vec, bos_token_id=0, max_length=max_length)
    return tokenizer.batch_decode(vgenout[:, 1:-1])[0]
enc_vec = extract_encoder_vector("我們<上>山途中欣賞沿途風景。", tokenizer, model)
decode_vector(enc_vec, tokenizer, model)

'VCL。往高處移動或去。'

In [24]:
enc_vec = extract_encoder_vector("在愛河邊施放五光十色的<煙火>", tokenizer, model)
decode_vector(enc_vec, tokenizer, model)

'Na。燃燒後所發出的煙。'

In [25]:
enc_vec = extract_encoder_vector("在愛河邊施放五光十色的<煙>火", tokenizer, model)
decode_vector(enc_vec, tokenizer, model)

'Na。燃燒後所產生的煙。'

In [26]:
enc_vec = extract_encoder_vector("在愛河邊施放五光十色的煙<火>", tokenizer, model)
decode_vector(enc_vec, tokenizer, model)

'Na。火的火。'

In [27]:
enc_vec = extract_encoder_vector("在愛河邊施放<五光十色>的煙火", tokenizer, model)
decode_vector(enc_vec, tokenizer, model)

'VH。形容光線充足且亮度高的。'

In [28]:
enc_vec = extract_encoder_vector("在<愛河>邊施放五光十色的煙火", tokenizer, model)
decode_vector(enc_vec, tokenizer, model)

'Nc。河川名,位於黃河注入湖海,為黃河注入湖海或其它河流的地方。'

In [30]:
from CwnGraph import CwnImage
cwn = CwnImage.latest()
cwn.find_senses(definition="光線充足")

[<CwnSense[03028901](明朗，VH): 形容光線充足明亮。>,
 <CwnSense[06649101](亮，VH): 形容光源的光線充足。>,
 <CwnSense[06685401](明，VH): 形容光源的光線充足。>,
 <CwnSense[07083901](朗，VH): 形容天空沒有烏雲遮蓋而光線充足明亮。>]

In [31]:
cwn.find_senses(definition="亮度高")

[<CwnSense[05227708](爛，VH): 形容顏色亮度高的。>,
 <CwnSense[06539512](清，VH): 形容比喻顏色較淺且亮度高的。>,
 <CwnSense[06649110](亮，VH): 形容比喻顏色較淺且亮度高的。>,
 <CwnSense[06682803](鮮，VH): 形容顏色亮度高的。>,
 <CwnSense[06685417](明，VH): 形容比喻顏色較淺且亮度高的。>,
 <CwnSense[07060506](粉，VH): 形容顏色亮度高，彩度不飽合的。>,
 <CwnSense[07067408](嫩，VH): 形容顏色亮度高，彩度不飽合的。>]

## Morphing vectors

In [20]:
enc_vec1 = extract_encoder_vector("我們<上>山途中欣賞沿途風景。", tokenizer, model)
enc_vec2 = extract_encoder_vector("我們在山裡<野餐>。", tokenizer, model)
delta = enc_vec2 - enc_vec1
for i in np.arange(0, 1.01, 0.2):
    print(f"{i:.2f}", decode_vector(enc_vec1+delta*i, tokenizer, model))

0.00 VCL。往高處移動或去。
0.20 VCL。往高處移動。
0.40 VCL。從參考位置的外面移到參考位置的裡面。
0.60 VCL。從原來所在地經過一段行程到其他地點,通常是山。
0.80 nom,VA。人們在約定俗成的固定時間內吃正餐。
1.00 nom,VA。人們在約定俗成的固定時間內吃正餐。


In [21]:
enc_vec1 = extract_encoder_vector("我們上山時，天<突然>下起了大雪。", tokenizer, model)
enc_vec2 = extract_encoder_vector("為什麼我的留言板是<空>的？", tokenizer, model)
delta = enc_vec2 - enc_vec1
for i in np.arange(0, 1.01, 0.2):
    print(f"{i:.2f}", decode_vector(enc_vec1+delta*i, tokenizer, model))

0.00 D。表突然出現在動詞之後。
0.20 D。表突然出現在腦海中。
0.40 D。表事件發生的頻率比預期低。
0.60 VH。形容比喻特定事件沒有被發生過。
0.80 VH。形容比喻特定對象沒有被使用。
1.00 VH。形容比喻特定版面變成沒有被填滿的狀態。


In [22]:
enc_vec1 = extract_encoder_vector("那是一位嬌豔如<花>的少女。", tokenizer, model)
enc_vec2 = extract_encoder_vector("以動畫方式慢速地顯示字母的每一<筆>一劃。", tokenizer, model)
delta = enc_vec2 - enc_vec1
for i in np.arange(0, 1.01, 0.2):
    print(f"{i:.2f}", decode_vector(enc_vec1+delta*i, tokenizer, model))

0.00 Na。植物的主要器官之一,主要用於繁殖,通常具有顏色鮮豔和形狀漂亮的花瓣。
0.20 Na。植物名,薔薇科櫻屬,多年生草本,葉呈長橢圓形,有花瓣五片,有粉紅、白、紅等顏色。
0.40 Na。植物名,薔薇科櫻屬,多年生草本,葉呈長橢圓形,有花瓣五片,有粉紅、白、紅等顏色。
0.60 Na。書寫筆畫的一種,由左向右上斜的筆畫。
0.80 Na。筆畫的一種,由左向右上斜的筆畫。
1.00 Nf。計算筆畫的單位。


In [23]:
enc_vec1 = extract_encoder_vector("昨天我的<貓>抓了三隻老鼠。", tokenizer, model)
enc_vec2 = extract_encoder_vector("昨天我的貓<抓>了三隻老鼠。", tokenizer, model)
delta = enc_vec2 - enc_vec1
for i in np.arange(0, 1.01, 0.2):
    print(f"{i:.2f}", decode_vector(enc_vec1+delta*i, tokenizer, model))

0.00 Na。哺乳類動物,瞳孔會因光線強弱而變大小,行動敏捷。
0.20 Na。以貓為形象製成的人造物。
0.40 Na。以貓為形象製成的人造物。
0.60 VC。比喻以不正當的方式取得財物。
0.80 VC。比喻以不正當的方式取得財物。
1.00 VC。調查或發現不合規範的行為。


In [34]:
enc_vec1 = extract_encoder_vector("<布丁狗>是三麗鷗今年票選第一名的角色。", tokenizer, model)
enc_vec2 = extract_encoder_vector("<凱蒂貓>有五個蘋果高三個蘋果重。", tokenizer, model)
delta = enc_vec2 - enc_vec1
for i in np.arange(0, 1.01, 0.2):
    print(f"{i:.2f}", decode_vector(enc_vec1+delta*i, tokenizer, model))

0.00 Nb。外文名字。
0.20 Nb。外文名字。
0.40 Nb。美國私立大學之一,位於美國東岸,為美國經濟、工商業中心。
0.60 Nb。美國私立大學之一,位於美國東岸,為美國經濟、工商業中心。
0.80 Na。伊拉克的文化。
1.00 Na。伊拉克的文化。


In [35]:
enc_vec1 = extract_encoder_vector("<中文詞彙網路>是台大語言所維護的語言資源之一。", tokenizer, model)
enc_vec2 = extract_encoder_vector("中文詞彙網路是<台大語言所>維護的語言資源之一。", tokenizer, model)
delta = enc_vec2 - enc_vec1
for i in np.arange(0, 1.01, 0.2):
    print(f"{i:.2f}", decode_vector(enc_vec1+delta*i, tokenizer, model))

0.00 Na。中華民族的主要通用語文,為中國大陸、臺灣、新加坡的官方語言。
0.20 Na。中華人民共和國的語文學系統,由中國國民黨創立。
0.40 Na。臺灣國立大學之一,位於臺北市文山區。
0.60 Nb。臺灣國立大學之一,位於臺北市文山區。
0.80 Nb。臺灣國立大學之一,位於臺北市大安區。
1.00 Nb。臺灣國立大學之一,位於臺北市大安區。


## More playground

In [49]:
def gen_func(tokenizer, model):
    def _gen_func(text):
        enc_vec = extract_encoder_vector(text, tokenizer, model)
        return decode_vector(enc_vec, tokenizer, model)
    return _gen_func
gen = gen_func(tokenizer, model)

In [55]:
gen("<中文詞彙網路是台大語言所維護的語言資源之一。>")

'Na。指由專業人員編排並提供授予碩士或博士學位的教育系統。'

In [101]:
enc_vec = extract_encoder_vector("在台大校慶典禮上，學生<扮裝>進場。。", tokenizer, model)
for _ in range(10):
    print(decode_vector(enc_vec+torch.randn(768)*0.05, tokenizer, model))

nom,VC。在服裝或服飾裝飾中,依照其美麗外貌,改變外表造型。
VC。以特定形式做出造型。
nom,VA。依特定對象的造型或頭部做造型。
nom,VA。依照特定的舉例做造型。
nom,VA。依照特定材料做造型以進行特定活動。
nom,VA。依照特定造型或造型更改造型。
nom,VC。改變造型或外表造型。
nom,VC。依照特定的形狀或立場將特定對象的性質或狀態改變。
nom,VA。改變造型。
nom,VC。在正式場合或特定活動中擔任特定角色。
